# Setup

In [73]:
import os
from pprint import pprint
from typing import Iterable
from typing import List
from typing import Literal
from typing import Optional

from dotenv import find_dotenv
from dotenv import load_dotenv
import instructor
from openai import AzureOpenAI
from pydantic import BaseModel
from pydantic import Field


load_dotenv(find_dotenv())
DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")
client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-02-01",
)
client = instructor.patch(client, mode=instructor.Mode.FUNCTIONS)

In [47]:
def get_completion(
    prompt: str,
    response_model: BaseModel | Iterable[BaseModel],
    stream: bool = False,
    client: AzureOpenAI = client,
) -> BaseModel | Iterable[BaseModel]:
    return client.chat.completions.create(
        model=DEPLOYMENT,
        messages=[{"role": "user", "content": prompt}],
        response_model=response_model,
        temperature=0,
        stream=stream,
    )

# Classification

In [34]:
class BiblicalPersonLiteral(BaseModel):
    """A Person from the Bible"""

    name: str = Field(description="The full name")
    kingdom: Literal["Israel", "Judah"]


resp = get_completion("Elijah", BiblicalPersonLiteral)
resp

BiblicalPersonLiteral(name='Elijah', kingdom='Israel')

In [33]:
from enum import Enum


class Kingdom(Enum):
    Israel = "Israel"
    Judah = "Judah"


class BiblicalPersonEnum(BaseModel):
    """A Person from the Bible"""

    name: str = Field(description="The full name")
    kingdom: Kingdom


resp = get_completion("Elijah", BiblicalPersonEnum)
resp

BiblicalPersonEnum(name='Elijah', kingdom=<Kingdom.Israel: 'Israel'>)

# Arbitrary properties

In [35]:
class Property(BaseModel):
    key: str
    value: str


class BiblicalPersonProperties(BiblicalPersonLiteral):
    properties: List[Property]

In [36]:
resp = get_completion("Elijah", BiblicalPersonProperties)

pprint(resp.model_dump())

{'kingdom': 'Israel',
 'name': 'Elijah',
 'properties': [{'key': 'Occupation', 'value': 'Prophet'},
                {'key': 'Lifespan', 'value': '9th century BC'},
                {'key': 'Notable Events',
                 'value': 'Confrontation with Ahab, contest with the prophets '
                          'of Baal, being taken up to heaven in a whirlwind'},
                {'key': 'Associated Miracles',
                 'value': "Causing a drought, raising the widow's son, calling "
                          'down fire from heaven'},
                {'key': 'Mentor', 'value': 'God'},
                {'key': 'Successor', 'value': 'Elisha'}]}


## Limited amount of properties

In [37]:
class PropertyWithId(BaseModel):
    index: str = Field(description="Monotonically increasing int starting from 1")
    key: str
    value: str


class BiblicalPersonLimitedProperties(BiblicalPersonLiteral):
    properties: List[PropertyWithId] = Field(
        description="Numbered list of properties. Max length is 3"
    )

In [38]:
resp = get_completion("Elijah", BiblicalPersonLimitedProperties)

pprint(resp.model_dump())

{'kingdom': 'Israel',
 'name': 'Elijah',
 'properties': [{'index': '1',
                 'key': 'Prophet',
                 'value': 'Elijah was a prophet in the northern kingdom of '
                          'Israel during the reign of Ahab.'},
                {'index': '2',
                 'key': 'Miracles',
                 'value': 'He performed miracles, including raising the dead, '
                          'bringing fire down from the sky, and causing a '
                          'drought.'},
                {'index': '3',
                 'key': 'Contest at Mount Carmel',
                 'value': 'Elijah challenged the prophets of Baal to a contest '
                          'at Mount Carmel to prove that the Lord is the true '
                          'God.'}]}


# List of entities

In [57]:
from time import monotonic

In [67]:
resp = get_completion(
    "The prophets of the Old Testament", Iterable[BiblicalPersonLiteral], stream=True
)

t1 = monotonic()
for i in resp:
    t2 = monotonic()
    print(f"Time diff: {t2-t1:.1f}, entity: {i}")
    t1 = t2

Time diff: 0.7, entity: name='Isaiah' kingdom='Judah'
Time diff: 0.6, entity: name='Jeremiah' kingdom='Judah'
Time diff: 0.6, entity: name='Ezekiel' kingdom='Israel'
Time diff: 0.4, entity: name='Daniel' kingdom='Israel'
Time diff: 0.5, entity: name='Hosea' kingdom='Israel'
Time diff: 0.5, entity: name='Joel' kingdom='Judah'
Time diff: 0.5, entity: name='Amos' kingdom='Israel'
Time diff: 0.5, entity: name='Obadiah' kingdom='Judah'
Time diff: 0.5, entity: name='Jonah' kingdom='Israel'
Time diff: 0.5, entity: name='Micah' kingdom='Judah'
Time diff: 0.8, entity: name='Nahum' kingdom='Israel'
Time diff: 1.0, entity: name='Habakkuk' kingdom='Judah'
Time diff: 1.1, entity: name='Zephaniah' kingdom='Judah'
Time diff: 0.7, entity: name='Haggai' kingdom='Judah'
Time diff: 1.3, entity: name='Zechariah' kingdom='Judah'
Time diff: 0.6, entity: name='Malachi' kingdom='Judah'


# Relationships

In [72]:
class BiblicalPersonWithRelationships(BaseModel):
    id: int
    name: str
    enemies: List[int] = Field(description="Relationships to the enemies and opponents")


resp = get_completion(
    "Elijah, John the Baptist, Ahab, Herod Antipas",
    Iterable[BiblicalPersonWithRelationships],
    stream=True,
)

t1 = monotonic()
for i in resp:
    t2 = monotonic()
    print(f"Time diff: {t2-t1:.1f}\n{i}\n")
    t1 = t2

Time diff: 1.0
id=1 name='Elijah' enemies=[2]

Time diff: 2.0
id=2 name='Ahab' enemies=[1]

Time diff: 1.0
id=3 name='John the Baptist' enemies=[4]

Time diff: 1.1
id=4 name='Herod Antipas' enemies=[3]



# Hallucination handling

In [87]:
class OptionPerson(BaseModel):
    result: Optional[BiblicalPersonLiteral] = Field(default=None)
    error: bool = Field(
        default=False, description="`True` if you didn't manage to find a person from the Bible"
    )
    message: Optional[str] = Field(
        description="Description of an error if a person was not parsed", default=None
    )

In [83]:
resp = get_completion("Elijah", OptionPerson)

resp

OptionPerson(result=BiblicalPersonLiteral(name='Elijah', kingdom='Israel'), error=False, message=None)

In [88]:
resp = get_completion("Minecraft lore", OptionPerson)

resp

OptionPerson(result=None, error=True, message='The request does not pertain to a Biblical person.')